In [1]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.market import Market
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from tqdm import *
import wandb

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
LOG_DIR = '.'
trn_dataset = dataset.load('market', '../../Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

In [4]:
dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = 50,
        shuffle = True,
        num_workers = 4,
        drop_last = True,
        pin_memory = True
    )

In [5]:
model = Resnet50(embedding_size=512, pretrained=True, is_norm=1, bn_freeze =1).cuda()

/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = losses.Proxy_Anchor(nb_classes = trn_dataset.nb_classes(), sz_embed = 512, mrg = 0.1, alpha = 32).cuda()

In [7]:
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters())))},
    {'params': model.model.embedding.parameters(), 'lr':float(1e-4) * 1},
]
param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

In [8]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [9]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [10]:
for epoch in range(0, 60):
    model.train()
    bn_freeze = True

    if bn_freeze:
            modules = model.model.modules()
            for m in modules: 
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()

    losses_per_epoch = []
    unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(dl_tr))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))

0it [00:00, ?it/s]/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Train Epoch: 0 [44/44 (98%)] Loss: 7.966811: : 44it [00:04,  9.33it/s] 
Train Epoch: 1 [44/44 (98%)] Loss: 7.491183: : 44it [00:11,  3.71it/s]
Train Epoch: 2 [44/44 (98%)] Loss: 7.233761: : 44it [00:11,  3.74it/s]
Train Epoch: 3 [44/44 (98%)] Loss: 6.655641: : 44it [00:11,  3.74it/s]
Train Epoch: 4 [44/44 (98%)] Loss: 4.952507: : 44it [00:11,  3.74it/s]
Train Epoch: 5 [44/44 (98%)] Loss: 5.777985: : 44it [00:11,  3.74it/s]
Train Epoch: 6 [44/44 (98%)] Loss: 4.920753: : 44it [00:11,  3.74it/s]
Train Epoch: 7 [44/44 (98%)] Loss: 4.813202: : 44it [00:11,  3.74it/s]
Train Epoch: 8 [44/44 (98%)] Loss: 4.158882: : 44it [00:11,  3.75it/s]
Train Epoch: 9 [44/

In [11]:
model

Resnet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [13]:
eval_dataset = dataset.load('market', '../../Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = False, 
                                                                                                                        is_inception = False))

In [37]:
from dataset.base import BaseDataset

class Market(BaseDataset):
    def __init__(self, root, mode, transform = None):
        self.root = root
        self.mode = mode

        gt_bbox = [os.path.join(self.root, 'gt_bbox', x) for x in os.listdir(os.path.join(self.root, 'gt_bbox'))]
        bbox_train = [os.path.join(self.root, 'bounding_box_train', x) for x in os.listdir(os.path.join(self.root, 'bounding_box_train'))]
        self.train_images = gt_bbox + bbox_train
        
        self.transform = transform
        if self.mode == 'train':
            self.classes = range(0,100)
        elif self.mode == 'eval':
            self.classes = range(1000,1501)
                
        BaseDataset.__init__(self, self.root, self.mode, self.transform)
        
        ys = [int(image.split('/')[-1].split('_')[0]) for image in self.train_images]
        index = 0
        self.im_paths = []
        for im_path, y in zip(self.train_images, ys):
            if y in self.classes: # choose only specified classes
                self.im_paths.append(im_path)
                self.ys.append(y)
                self.I += [index]
                index += 1

    def set_one_class(self, class_label):
        
        self.ys = []
        self.im_paths = []
        self.I = []
        ys = [int(image.split('/')[-1].split('_')[0]) for image in self.train_images]
        index = 0
        for im_path, y in zip(self.train_images, ys):
            if y ==class_label: # choose only specified classes
                self.im_paths.append(im_path)
                self.ys.append(y)
                self.I += [index]
                index += 1
    def reset(self):
        self.__init__(self.root, self.mode, self.transform)

In [38]:
d = Market('../../Market-1501-v15.09.15/', 'eval', dataset.utils.make_transform(is_train = False, is_inception = False))

In [68]:
d.set_one_class(1070)

In [69]:
model.eval()
for images, labels in dl_ev:
    embs_pred_2 = model(images.squeeze().cuda())

In [79]:
d.set_one_class(1200)

dl_ev = torch.utils.data.DataLoader(
        d,
        batch_size = len(d),
        shuffle = True,
        num_workers = 4,
        drop_last = True,
        pin_memory = True
)
model.eval()
for images, labels in dl_ev:
    embs_pred_1 = model(images.squeeze().cuda())

In [81]:
F.linear(embs_pred_1[0].reshape(1, 512), embs_pred_1)

tensor([[1.0000, 0.8198, 0.8986, 0.5389, 0.4660, 0.8786, 0.5574, 0.4878, 0.7150,
         0.7501, 0.4738, 0.8181, 0.6980, 0.7948, 0.5061, 0.6141, 0.5627, 0.5643,
         0.5660, 0.4125, 0.4872, 0.5157, 0.4867, 0.8246, 0.5339, 0.4964, 0.5338,
         0.7170, 0.8414, 0.7112, 0.8509]], device='cuda:0',
       grad_fn=<MmBackward0>)